# Workshop - Analysis of Geometric Shapes: Centroid, Area, and Perimeter

## Load binary image

In [1]:
import cv2

img = cv2.imread('../data/Lenna.png')
assert img is not None, 'Failed to load image'

# Convert to grayscale
img_gray = cv2.cvtColor(
    img,
    cv2.COLOR_BGR2GRAY,
)


# Threshold the image
_, thresh = cv2.threshold(
    img_gray,
    127,
    255,
    cv2.THRESH_BINARY,
)

thresh

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], shape=(250, 250), dtype=uint8)

## Find contours

In [2]:
contours, hierarchy = cv2.findContours(
    thresh,
    cv2.RETR_TREE, # Retrieves all contours and reconstructs a full hierarchy of nested contours
    cv2.CHAIN_APPROX_SIMPLE, # Reduces the number of points in the contour
)

print(f'Number of contours found: {len(contours)}')

Number of contours found: 252
